# Carga de los Datos y Preprocesamiento

In [20]:
# Carga de librerías
import pandas as pd

# Cargar la base de datos
df = pd.read_parquet('train202212_202307.parquet')

# Selección de las variables requeridas para entrenar el modelo
X = df[['DiasAtraso','TipoPrestamo', 'Relacion_coutasPagasPlazo', 'Relacion_pagoTotal', 'Genero', 'TipoVivienda', 
        'LocalizacionNegocio', 'Edad', 'EstadoCivil', 'Hijos', 'PersonasCargo','Calificacion_ult_gestion',
        'Calificacion_ult_gestion_3_meses', 'Calificacion_ult_gestion_6_meses', 'Recuento Incumplido 3 meses',
        'Recuento Incumplido 6 meses', 'Recuento con contacto ultimos 3 meses', 'Recuento con contacto ultimos 6 meses',
        'Recuento_sin_contacto_ultimos_3_meses', 'Recuento_sin_contacto_ultimos_6_meses']]

# Variable objetivo
y = df["P_recuperacionF"]

# One-hot encoding para variables categóricas
X_encoded = pd.get_dummies(X, columns=['TipoPrestamo', 'Genero', 'TipoVivienda', 'LocalizacionNegocio', 'EstadoCivil'], drop_first=True)



In [56]:
X_encoded.columns

Index(['DiasAtraso', 'Relacion_coutasPagasPlazo', 'Relacion_pagoTotal', 'Edad',
       'Hijos', 'PersonasCargo', 'Calificacion_ult_gestion',
       'Calificacion_ult_gestion_3_meses', 'Calificacion_ult_gestion_6_meses',
       'Recuento Incumplido 3 meses', 'Recuento Incumplido 6 meses',
       'Recuento con contacto ultimos 3 meses',
       'Recuento con contacto ultimos 6 meses',
       'Recuento_sin_contacto_ultimos_3_meses',
       'Recuento_sin_contacto_ultimos_6_meses', 'TipoPrestamo_C',
       'TipoPrestamo_D', 'TipoPrestamo_F', 'TipoPrestamo_M', 'TipoPrestamo_N',
       'TipoPrestamo_O', 'TipoPrestamo_R', 'TipoPrestamo_S', 'TipoPrestamo_X',
       'Genero_M', 'Genero_ND', 'TipoVivienda_A', 'TipoVivienda_F',
       'TipoVivienda_N', 'TipoVivienda_O', 'TipoVivienda_P',
       'LocalizacionNegocio_D', 'LocalizacionNegocio_I',
       'LocalizacionNegocio_R', 'EstadoCivil_ ', 'EstadoCivil_A',
       'EstadoCivil_C', 'EstadoCivil_D', 'EstadoCivil_S', 'EstadoCivil_V'],
      dtype='ob

# División de los Datos

In [21]:
# Importar librerías de Scikit-learn
from sklearn.model_selection import train_test_split

# División de los datos en entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)


In [61]:
X_encoded.columns

Index(['DiasAtraso', 'Relacion_coutasPagasPlazo', 'Relacion_pagoTotal', 'Edad',
       'Hijos', 'PersonasCargo', 'Calificacion_ult_gestion',
       'Calificacion_ult_gestion_3_meses', 'Calificacion_ult_gestion_6_meses',
       'Recuento Incumplido 3 meses', 'Recuento Incumplido 6 meses',
       'Recuento con contacto ultimos 3 meses',
       'Recuento con contacto ultimos 6 meses',
       'Recuento_sin_contacto_ultimos_3_meses',
       'Recuento_sin_contacto_ultimos_6_meses', 'TipoPrestamo_C',
       'TipoPrestamo_D', 'TipoPrestamo_F', 'TipoPrestamo_M', 'TipoPrestamo_N',
       'TipoPrestamo_O', 'TipoPrestamo_R', 'TipoPrestamo_S', 'TipoPrestamo_X',
       'Genero_M', 'Genero_ND', 'TipoVivienda_A', 'TipoVivienda_F',
       'TipoVivienda_N', 'TipoVivienda_O', 'TipoVivienda_P',
       'LocalizacionNegocio_D', 'LocalizacionNegocio_I',
       'LocalizacionNegocio_R', 'EstadoCivil_ ', 'EstadoCivil_A',
       'EstadoCivil_C', 'EstadoCivil_D', 'EstadoCivil_S', 'EstadoCivil_V'],
      dtype='ob

# Modelo XGBoost - Base y GridSearchCV

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error

# Convertimos los datos a DMatrix (formato XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Definición del modelo base XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse', random_state=42)

# Parámetros a probar en el GridSearch
param_grid = {
    'max_depth': [6],
    'eta': [0.05],
    'subsample': [0.9],
    'colsample_bytree': [0.7],
    'n_estimators': [100, 200, 300]  # Número de árboles
}

# Configuración y ajuste del GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Mejor modelo
best_model = grid_search.best_estimator_

# Predicciones y evaluación
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE en conjunto de prueba: {rmse:.4f}")

Fitting 3 folds for each of 3 candidates, totalling 9 fits
RMSE en conjunto de prueba: 0.2697


  # Entrenamiento del Modelo XGBoost (Sin GridSearch)

In [23]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Entrenamiento directo del modelo XGBoost sin GridSearch
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 6,
    'eta': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.8
}

num_round = 300  # Número de iteraciones
model = xgb.train(params, dtrain, num_round)

# Predicción en el conjunto de prueba
y_pred = model.predict(dtest)

# Evaluación del modelo
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")


RMSE: 0.2696
MAE: 0.1540
R²: 0.2706


In [24]:
# Obtener la importancia de las características
importance = model.get_score(importance_type='weight')

# Ordenar características por importancia
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)

# Imprimir las 15 características más importantes
top_15_features = [feature for feature, score in sorted_importance[:15]]
print("\nLas 15 características más importantes son:")
for feature in top_15_features:
    print(feature)


Las 15 características más importantes son:
DiasAtraso
Edad
Relacion_pagoTotal
Relacion_coutasPagasPlazo
Recuento con contacto ultimos 3 meses
Calificacion_ult_gestion_3_meses
Hijos
Calificacion_ult_gestion
Recuento_sin_contacto_ultimos_3_meses
PersonasCargo
Recuento con contacto ultimos 6 meses
Calificacion_ult_gestion_6_meses
Recuento_sin_contacto_ultimos_6_meses
TipoPrestamo_X
LocalizacionNegocio_I


# Predicción Real (Datos de Test)

In [25]:
x_prediccion = pd.read_parquet('test_sep_2024F_cartera.parquet')

In [26]:
x_prediccion

,FechaCorte,Credito,P_recuperacion,P_recuperacionF,DiasAtraso,Saldo,MontoDesembolsado,TipoPrestamo,CuotasPagas,Cuotas,...,Numero De Obligacion,Calificacion_ult_gestion,Calificacion_ult_gestion_3_meses,Calificacion_ult_gestion_6_meses,Recuento Incumplido 3 meses,Recuento Incumplido 6 meses,Recuento con contacto ultimos 3 meses,Recuento con contacto ultimos 6 meses,Recuento_sin_contacto_ultimos_3_meses,Recuento_sin_contacto_ultimos_6_meses
0,2024-09-30,101151057692-1,0.0,0.0,2905,1001384.0,2150000.0,C,11,18.0,...,101151057692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-09-30,101151057696-1,0.0,0.0,3055,2469674.0,3000000.0,C,6,24.0,...,101151057696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-09-30,101151057708-1,0.0,0.0,2965,9008916.0,12800000.0,M,9,24.0,...,101151057708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-09-30,101151057710-1,0.0,0.0,3054,2654655.0,3500000.0,C,6,20.0,...,101151057710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-09-30,101151057752-1,0.0,0.0,2995,1031218.0,1630000.0,N,8,18.0,...,101151057752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
389794,2024-09-30,904201006642-1,0.0,0.0,1576,12901894.0,13150000.0,S,1,30.0,...,904201006642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
389795,2024-09-30,904201006650-1,0.0,0.0,1099,2429231.0,5194000.0,S,16,26.0,...,904201006650,2.0,2.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0
389796,2024-09-30,904201006699-1,0.0,0.0,1064,2465173.0,5155000.0,N,15,24.0,...,904201006699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
389797,2024-09-30,904201006717-1,0.0,0.0,1190,6582321.0,8471264.0,X,10,30.0,...,904201006717,3.0,2.0,2.0,0.0,0.0,8.0,1.0,2.0,4.0


In [27]:
x_prediccion.columns

Index(['FechaCorte', 'Credito', 'P_recuperacion', 'P_recuperacionF',
       'DiasAtraso', 'Saldo', 'MontoDesembolsado', 'TipoPrestamo',
       'CuotasPagas', 'Cuotas', 'Relacion_coutasPagasPlazo',
       'Relacion_pagoTotal', 'Genero', 'TipoVivienda', 'LocalizacionNegocio',
       'EstratoDomicilio', 'Edad', 'EstadoCivil', 'Hijos', 'PersonasCargo',
       'Numero De Obligacion', 'Calificacion_ult_gestion',
       'Calificacion_ult_gestion_3_meses', 'Calificacion_ult_gestion_6_meses',
       'Recuento Incumplido 3 meses', 'Recuento Incumplido 6 meses',
       'Recuento con contacto ultimos 3 meses',
       'Recuento con contacto ultimos 6 meses',
       'Recuento_sin_contacto_ultimos_3_meses',
       'Recuento_sin_contacto_ultimos_6_meses'],
      dtype='object')

# Calificación Real

In [28]:
import pandas as pd
import xgboost as xgb

# Cargar datos de test
x_prediccion = pd.read_parquet('test_sep_2024F_cartera.parquet')

# Selección de columnas para el conjunto de prueba (X_p)
X_p = x_prediccion[['DiasAtraso', 'TipoPrestamo', 'Relacion_coutasPagasPlazo', 'Relacion_pagoTotal', 'Genero', 
                    'TipoVivienda', 'LocalizacionNegocio', 'Edad', 'EstadoCivil', 'Hijos', 'PersonasCargo',
                    'Calificacion_ult_gestion', 'Calificacion_ult_gestion_3_meses', 'Calificacion_ult_gestion_6_meses', 
                    'Recuento Incumplido 3 meses', 'Recuento Incumplido 6 meses', 'Recuento con contacto ultimos 3 meses',
                    'Recuento con contacto ultimos 6 meses', 'Recuento_sin_contacto_ultimos_3_meses', 
                    'Recuento_sin_contacto_ultimos_6_meses']]

# One-hot encoding para las columnas categóricas
X_encoded_p = pd.get_dummies(X_p, columns=['TipoPrestamo', 'Genero', 'TipoVivienda', 'LocalizacionNegocio', 'EstadoCivil'], drop_first=True)

# Obtener las características principales (top_15_features debe estar definido previamente)
x_reducido_p = X_encoded_p[top_15_features]  # Asegúrate de que `top_15_features` esté definido correctamente

# Verifica las columnas en X_encoded_p y en el modelo
missing_cols = set(model.feature_names) - set(x_reducido_p.columns)
extra_cols = set(x_reducido_p.columns) - set(model.feature_names)

# Si hay columnas faltantes, añádelas con valores 0 (puedes ajustarlo según lo necesites)
for col in missing_cols:
    x_reducido_p[col] = 0  # Asigna 0 o el valor predeterminado apropiado

# Si hay columnas extras, elimínelas
x_reducido_p = x_reducido_p[model.feature_names]

# Convertir a DMatrix para XGBoost
dtest_p = xgb.DMatrix(x_reducido_p)

# Realizar las predicciones
y_pred_p = model.predict(dtest_p)

# Guardar los resultados en un archivo CSV
df_resultado = pd.DataFrame({
    'NumeroObligacion': x_prediccion['Numero De Obligacion'],
    'Recuperacion': y_pred_p
})

# Guardar el archivo CSV
df_resultado.to_csv('Prediccion_Recuperacion_final.csv', index=False)
print("Archivo guardado como 'Prediccion_Recuperacion_final.csv")


C:\Users\IVAN.LIPES\AppData\Local\Temp\ipykernel_23312\3694541622.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_reducido_p[col] = 0  # Asigna 0 o el valor predeterminado apropiado
C:\Users\IVAN.LIPES\AppData\Local\Temp\ipykernel_23312\3694541622.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_reducido_p[col] = 0  # Asigna 0 o el valor predeterminado apropiado
C:\Users\IVAN.LIPES\AppData\Local\Temp\ipykernel_23312\3694541622.py:27: SettingWithCopyWarning: 
A value is trying to be set on a cop

Archivo guardado como 'Prediccion_Recuperacion_final.csv'


In [29]:
X_p.columns

Index(['DiasAtraso', 'TipoPrestamo', 'Relacion_coutasPagasPlazo',
       'Relacion_pagoTotal', 'Genero', 'TipoVivienda', 'LocalizacionNegocio',
       'Edad', 'EstadoCivil', 'Hijos', 'PersonasCargo',
       'Calificacion_ult_gestion', 'Calificacion_ult_gestion_3_meses',
       'Calificacion_ult_gestion_6_meses', 'Recuento Incumplido 3 meses',
       'Recuento Incumplido 6 meses', 'Recuento con contacto ultimos 3 meses',
       'Recuento con contacto ultimos 6 meses',
       'Recuento_sin_contacto_ultimos_3_meses',
       'Recuento_sin_contacto_ultimos_6_meses'],
      dtype='object')

In [30]:
df_resultado

,NumeroObligacion,Recuperacion
0,101151057692,0.034791
1,101151057696,0.039545
2,101151057708,0.021167
3,101151057710,0.025427
4,101151057752,0.019224
...,...,...
389794,904201006642,0.044193
389795,904201006650,0.272002
389796,904201006699,0.161777
389797,904201006717,0.473266


In [65]:
X_train

,DiasAtraso,Relacion_coutasPagasPlazo,Relacion_pagoTotal,Edad,Hijos,PersonasCargo,Calificacion_ult_gestion,Calificacion_ult_gestion_3_meses,Calificacion_ult_gestion_6_meses,Recuento Incumplido 3 meses,...,TipoVivienda_P,LocalizacionNegocio_D,LocalizacionNegocio_I,LocalizacionNegocio_R,EstadoCivil_,EstadoCivil_A,EstadoCivil_C,EstadoCivil_D,EstadoCivil_S,EstadoCivil_V
162196,534,0.750000,0.228649,25.0,0,0,0.0,1.0,0.0,0.0,...,False,False,True,False,False,False,False,False,True,False
157641,1510,0.500000,0.553709,27.0,0,0,0.0,0.0,0.0,0.0,...,False,False,True,False,False,False,False,False,True,False
79190,951,0.000000,1.000000,33.0,0,0,0.0,3.0,3.0,0.0,...,False,False,False,False,False,False,False,False,True,False
838,2815,0.041666,0.973061,33.0,1,1,0.0,1.0,0.0,0.0,...,False,False,True,False,False,True,False,False,False,False
63814,2725,0.875000,0.174179,40.0,1,0,0.0,1.0,0.0,0.0,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,950,0.000000,1.000000,59.0,6,1,1.0,1.0,1.0,0.0,...,True,False,False,False,False,False,False,True,False,False
103694,577,0.194444,0.803387,39.0,2,3,0.0,2.0,2.0,0.0,...,False,False,True,False,False,False,True,False,False,False
131932,2542,0.791666,0.283255,64.0,0,0,0.0,0.0,0.0,0.0,...,True,False,False,False,False,False,False,True,False,False
146867,1709,0.375000,0.720970,36.0,1,1,0.0,0.0,0.0,0.0,...,True,False,False,True,False,False,True,False,False,False


In [66]:
X_train_scaled = scaler.fit_transform(X_train)

In [67]:
X_train_scaled

array([[-0.86382973,  1.75089332, -2.12763596, ..., -0.43233095,
         1.70583916, -0.18318395],
       [ 0.04305018,  0.74493114, -0.7497327 , ..., -0.43233095,
         1.70583916, -0.18318395],
       [-0.47636157, -1.26699322,  1.14205734, ..., -0.43233095,
         1.70583916, -0.18318395],
       ...,
       [ 1.00196419,  1.918551  , -1.89616297, ...,  2.31304281,
        -0.58622174, -0.18318395],
       [ 0.22795705,  0.24195005, -0.04072772, ..., -0.43233095,
        -0.58622174, -0.18318395],
       [-1.03758643, -0.26103104,  0.40315081, ..., -0.43233095,
         1.70583916, -0.18318395]])

In [68]:
# Verificar si hay valores NaN o infinitos en los datos
import numpy as np
print(np.isnan(X_train_scaled).sum())  # Verifica NaN en X_train_scaled
print(np.isinf(X_train_scaled).sum())  # Verifica inf en X_train_scaled

4
0


In [73]:
from sklearn.impute import SimpleImputer

# Imputar los NaN con la media de cada columna
imputer = SimpleImputer(strategy='median')
X_train_scaled_imputed = imputer.fit_transform(X_train_scaled)

# Verificar si hay NaN después de la imputación
print(np.isnan(X_train_scaled_imputed).sum())  # Verifica NaN en X_train_scaled


0


In [74]:
print("Dimensiones de X_train_scaled:", X_train_scaled.shape)
print("Dimensiones de y_train:", y_train.shape)


Dimensiones de X_train_scaled: (161580, 40)
Dimensiones de y_train: (161576,)


In [77]:
import numpy as np

# Verificar dimensiones antes de eliminar NaN
print("Dimensiones de X_train_scaled:", X_train_scaled.shape)
print("Dimensiones de y_train:", y_train.shape)

# Recortar y_train para que tenga el mismo número de filas que X_train_scaled
# Asegúrate de que ambas matrices tengan el mismo número de filas
min_rows = min(X_train_scaled.shape[0], y_train.shape[0])
X_train_scaled = X_train_scaled[:min_rows]
y_train = y_train[:min_rows]

# Verificar nuevamente las dimensiones después del ajuste
print("Dimensiones de X_train_scaled después del ajuste:", X_train_scaled.shape)
print("Dimensiones de y_train después del ajuste:", y_train.shape)

# Eliminar filas que contienen NaN en X_train_scaled y en y_train
valid_indices = ~np.isnan(X_train_scaled).any(axis=1) & ~np.isnan(y_train)
X_train_scaled = X_train_scaled[valid_indices]
y_train = y_train[valid_indices]

# Verificar que no haya NaN después de eliminar las filas
print(np.isnan(X_train_scaled).sum())  # Verifica que no haya NaN en X_train_scaled
print(np.isnan(y_train).sum())  # Verifica que no haya NaN en y_train


Dimensiones de X_train_scaled: (161572, 40)
Dimensiones de y_train: (161572,)
Dimensiones de X_train_scaled después del ajuste: (161572, 40)
Dimensiones de y_train después del ajuste: (161572,)
0
0


In [39]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Crear y entrenar el modelo de KNN
knn_model = KNeighborsRegressor(n_neighbors=10)  # Puedes ajustar el número de vecinos (k)
knn_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_knn = knn_model.predict(X_test_scaled)

# Evaluar el modelo con RMSE
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))

print(f"RMSE del modelo KNN: {rmse_knn:.4f}")


RMSE del modelo KNN: 0.2795


In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Crear y entrenar el modelo de regresión lineal
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_lr = lr_model.predict(X_test_scaled)

# Evaluar el modelo con RMSE
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))

print(f"RMSE del modelo de Regresión Lineal: {rmse_lr:.4f}")


RMSE del modelo de Regresión Lineal: 0.2901


# Árboles de Decisión (Decision Tree Regressor)

In [41]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Crear y entrenar el modelo de árbol de decisión
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_dt = dt_model.predict(X_test_scaled)

# Evaluar el modelo con RMSE
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))

print(f"RMSE del modelo Árbol de Decisión: {rmse_dt:.4f}")


RMSE del modelo Árbol de Decisión: 0.3415


# Regresión Lasso (Lasso Regression)

In [42]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import numpy as np

# Crear y entrenar el modelo de regresión Lasso
lasso_model = Lasso(alpha=0.1)  # Ajusta alpha para controlar la regularización
lasso_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_lasso = lasso_model.predict(X_test_scaled)

# Evaluar el modelo con RMSE
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))

print(f"RMSE del modelo Lasso: {rmse_lasso:.4f}")


RMSE del modelo Lasso: 0.3157


# Random Forest Regressor

In [78]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import numpy as np

# Crear y entrenar el modelo de Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test_scaled)

# Evaluar el modelo con RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_1 = mean_absolute_error(y_test, y_pred)
r2_1 = r2_score(y_test, y_pred)
print(f"RMSE del modelo Random Forest: {rmse_rf:.4f}")
print(f"MAE del modelo Random Forest: {mae_1:.4f}")
print(f"r2 del modelo Random Forest: {r2_1:.4f}")

RMSE del modelo Random Forest: 0.2998
MAE del modelo Random Forest: 0.1540
r2 del modelo Random Forest: 0.2706


In [54]:
X_train_scaled

array([[-0.86382973,  1.75089332, -2.12763596, ..., -0.43233095,
         1.70583916, -0.18318395],
       [ 0.04305018,  0.74493114, -0.7497327 , ..., -0.43233095,
         1.70583916, -0.18318395],
       [-0.47636157, -1.26699322,  1.14205734, ..., -0.43233095,
         1.70583916, -0.18318395],
       ...,
       [ 1.00196419,  1.918551  , -1.89616297, ...,  2.31304281,
        -0.58622174, -0.18318395],
       [ 0.22795705,  0.24195005, -0.04072772, ..., -0.43233095,
        -0.58622174, -0.18318395],
       [-1.03758643, -0.26103104,  0.40315081, ..., -0.43233095,
         1.70583916, -0.18318395]])

In [79]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Cargar los datos de test
x_prediccion = pd.read_parquet('test_sep_2024F_cartera.parquet')

# Preprocesar las características
X_p = x_prediccion[['DiasAtraso','TipoPrestamo', 'Relacion_coutasPagasPlazo', 'Relacion_pagoTotal', 'Genero', 'TipoVivienda', 
                     'LocalizacionNegocio', 'EstadoCivil', 'Hijos', 'PersonasCargo', 'Calificacion_ult_gestion',
                     'Calificacion_ult_gestion_3_meses', 'Calificacion_ult_gestion_6_meses', 'Recuento Incumplido 3 meses',
                     'Recuento Incumplido 6 meses', 'Recuento con contacto ultimos 3 meses', 'Recuento con contacto ultimos 6 meses',
                     'Recuento_sin_contacto_ultimos_3_meses', 'Recuento_sin_contacto_ultimos_6_meses']]

# Codificar las variables categóricas
X_encoded_p = pd.get_dummies(X_p, columns=['TipoPrestamo', 'Genero', 'TipoVivienda', 'LocalizacionNegocio', 'EstadoCivil'], drop_first=True)


X_reducido_p = X_encoded_p 

# Escalar las características (si es necesario)
scaler = StandardScaler()
X_scaled_p = scaler.fit_transform(X_reducido_p)

# Realizar las predicciones con el modelo Random Forest previamente entrenado
y_pred_rf = rf_model.predict(X_scaled_p)

# Crear un DataFrame con los resultados
df_resultado = pd.DataFrame({
    'NumeroObligacion': x_prediccion['Numero De Obligacion'],  # Asegúrate de que 'NumeroObligacion' esté presente en el DataFrame
    'Recuperacion': y_pred_rf
})

# Guardar las predicciones en un archivo CSV
df_resultado.to_csv('Prediccion_Recuperacion_final.csv', index=False)

print("Archivo CSV guardado como 'Prediccion_Recuperacion.csv'")


Archivo CSV guardado como 'Prediccion_Recuperacion.csv'


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import numpy as np

# Crear y entrenar el modelo de Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = rf_model.predict(X_test_scaled)

# Evaluar el modelo con RMSE
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_1 = mean_absolute_error(y_test, y_pred)
r2_1 = r2_score(y_test, y_pred)
print(f"RMSE del modelo Random Forest: {rmse_rf:.4f}")
print(f"MAE del modelo Random Forest: {mae_1:.4f}")
print(f"r2 del modelo Random Forest: {r2_1:.4f}")